In [ ]:
from tensorflow import keras
import keras.backend as K
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Conv3D, DepthwiseConv2D, SeparableConv2D, Conv3DTranspose
from keras.layers import Flatten, MaxPool2D, AvgPool2D, GlobalAvgPool2D, UpSampling2D, BatchNormalization
from keras.layers import Concatenate, Add, Dropout, ReLU, Lambda, Activation, LeakyReLU, PReLU
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from time import time
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
def append_ext(fn):
    return fn+".jpg"

#loading dataset 
df = pd.read_csv("dataset.csv")

df["img"] = df["img"].apply(append_ext)

df = df.replace(np.nan, 0)

columns = ['gummy','golden_proportion ','gap','crowding','incisal embrassure ','color','Central line','Crooked','Central Incisor W/H Ratio','Black triangle','perfect']



In [ ]:
#shuffle dataframe before splitting 
df=df.sample(frac=1).reset_index(drop=True)

In [ ]:
# # preprocessing the training set
# train_data_generation = ImageDataGenerator(rescale=1./255)

# training_set = train_data_generation.flow_from_dataframe(dataframe=traindf,
#                                                          directory="./cropped_teeth",
#                                                          x_col="img",
#                                                          y_col=columns,
#                                                          target_size=(224, 224),
#                                                          batch_size=1,
#                                                          class_mode="raw",seed=0)
# # preprocessing the testing set
# valid_data_generation = ImageDataGenerator(rescale=1./255)

# valid_set = valid_data_generation.flow_from_dataframe(dataframe=validdf,
#                                                          directory="./cropped_teeth",
#                                                          x_col="img",
#                                                          y_col=columns,
#                                                          target_size=(224, 224),
#                                                          batch_size=1,
#                                                          class_mode="raw",seed=0)

# # preprocessing the testing set
# test_data_generation = ImageDataGenerator(rescale=1./255)

# test_set = test_data_generation.flow_from_dataframe(dataframe=testdf,
#                                                          directory="./cropped_teeth",
#                                                          x_col="img",
#                                                          y_col=None,
#                                                          target_size=(224, 224),
#                                                          batch_size=1,
#                                                          class_mode=None,seed=0)

In [ ]:
# preprocessing the training set
train_data_generation = ImageDataGenerator(rescale=1./255)

training_set = train_data_generation.flow_from_dataframe(dataframe=df[:155],
                                                         directory="./cropped_teeth",
                                                         x_col="img",
                                                         y_col=columns,
                                                         target_size=(224, 224),
                                                         batch_size=1,
                                                         class_mode="raw",seed=0)
# preprocessing the testing set
valid_data_generation = ImageDataGenerator(rescale=1./255)

valid_set = valid_data_generation.flow_from_dataframe(dataframe=df[155:175],
                                                         directory="./cropped_teeth",
                                                         x_col="img",
                                                         y_col=columns,
                                                         target_size=(224, 224),
                                                         batch_size=1,
                                                         class_mode="raw",seed=0)

# preprocessing the testing set
test_data_generation = ImageDataGenerator(rescale=1./255)

test_set = test_data_generation.flow_from_dataframe(dataframe=df[175:],
                                                         directory="./cropped_teeth",
                                                         x_col="img",
                                                         y_col=None,
                                                         target_size=(224, 224),
                                                         batch_size=1,
                                                         class_mode=None,seed=0)

In [ ]:
def vgg(input_shape):
  
  input = Input(input_shape)
  
  x = Conv2D(64, 3, padding='same', activation='relu')(input)
  x = Conv2D(64, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = Conv2D(128, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = Conv2D(256, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = Conv2D(512, 3, padding='same', activation='relu')(x)
  x = MaxPool2D(2, strides=2, padding='same')(x)
  
  x = Flatten()(x)
  x = Dense(4096, activation='relu')(x)
  x = Dense(4096, activation='relu')(x)
  
  output = Dense(1000, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [ ]:
model= vgg([224,224,3])
model.summary()

In [ ]:
def mobilenet(input_shape):
  
  def mobilenet_block(x, f, s=1):
    x = DepthwiseConv2D(3, strides=s, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(f, 1, strides=1, padding='same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    return x
    
    
  input = Input(input_shape)

  x = Conv2D(32, 3, strides=2, padding='same')(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)

  x = mobilenet_block(x, 64)
  x = mobilenet_block(x, 128, 2)
  x = mobilenet_block(x, 128)

  x = mobilenet_block(x, 256, 2)
  x = mobilenet_block(x, 256)

  x = mobilenet_block(x, 512, 2)
  for i in range(5):
    x = mobilenet_block(x, 512)

  x = mobilenet_block(x, 1024, 2)
  x = mobilenet_block(x, 1024)
  
  x = GlobalAvgPool2D()(x)
  
  output = Dense(1000, activation='softmax')(x)
  
  model = Model(input, output)
  return model

In [ ]:
model2= mobilenet([224,224,3])
model2.summary()

In [ ]:
# vgg
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.fit(x=training_set, validation_data=valid_set, epochs=20)


In [ ]:
#mobilenet
model2.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model2.fit(x=training_set, validation_data=testing_set, epochs=20)